# California PPE Projections 
## Editable View
This document provides direct access to the underlying data informing the projections. In addition to using sliders to adjust stockpile days, you can edit burn rate assumptions and rerun the model on the spot.

We provide detailed annotations of how to adjust model parameters for readers that might not have experience in programming or the Python programming language. However, if you do have a programming background, we expose our source code here for even more interactivity and editability.

## Importing Python packages and defining basic interactivity functions

In [332]:
import os
import sys
import qgrid
import pathlib
import restart
import numpy as np
import pandas as pd
from ipywidgets import widgets
from restart import RestartModel
from restart import Data
from restart_datasets import data
from IPython.display import Javascript, display
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from restart_datasets import data
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

qgrid.set_grid_option('forceFitColumns', False)
PATH = pathlib.Path(restart.__file__).parent.absolute()
chart_colors = ["#77AADD", "#99DDFF", "#44BB99", "#BBCC33", "#AAAA00", "#EEDD88",
                "#EE8866", "#FFAABB", "#DDDDDD", "#000000"]

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, \
                        IPython.notebook.ncells())'))
    
def create_run_button(description):
    button = widgets.Button(description=description)
    button.on_click(run_all)
    return button

def generate_pie_chart(df, title="", show_decimal=False):
    fig=plt.figure(title=title)
    
    pie_chart = plt.pie(sizes=df.values.tolist(),
                        labels=df.index.values.tolist(),
                        display_labels="outside",
                        colors=chart_colors[:df.index.values.size],
                        display_values=True)
    if not show_decimal:
        pie_chart.values_format = "0"
    return fig

def generate_bar(df, title="", scientific_notation=False, small_xlabel=False):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    bar_chart = plt.bar(x=x_titles,
                        y=df,
                        colors=chart_colors[:df.index.values.size])
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_group_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="grouped",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_scatter(df, title="", scientific_notation=False, small_xlabel=True):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    scatter = plt.scatter(x=x_titles, y=df)
    
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig
    
def generate_stacked_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="stacked",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_separate_bar_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    bar_list = []
    for col in df.columns: # .values.tolist()
        bar_list.append(generate_bar(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return bar_list

def generate_separate_scatter_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    scatter_list = []
    for col in df.columns: # .values.tolist()
        scatter_list.append(generate_scatter(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return scatter_list

def generate_html_legend(df, colors=chart_colors, table=True, font_size=13):
    name = df.index.name.rsplit(" ", 1)[0]
    html_string = f"<div style='font-size:{font_size}px; font-family:helvetica'><b style='font-weight:bold'>{name}</b>"
    indices = df.index.values
    if table:
        html_string += "<table><tr>"
        for i in range(0, indices.size):
            if i % 2 == 0:
                index_num = int(i / 2)
            else:
                index_num = int(i / 2 + indices.size / 2)
            html_string += f"<td style='padding:0 5px'><span style='color:{colors[index_num]}'>█</span> {indices[index_num]}</td>"
            if i % 2 != 0:
                html_string += "</tr><tr>"
        html_string += "</tr></table>"
    else:
        for_count = 0
        for string in indices:
            if for_count == 0:
                html_string += "<br>"
            else:
                html_string += "&emsp;"
            html_string += f"<span style='color:{colors[for_count]}'>█</span> {string}"
            for_count += 1
    html_string += "</div>"
    return widgets.HTML(html_string)

def generate_group_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_group_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

def generate_stacked_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_stacked_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

## Generating model

In [333]:
config = set_config(os.path.join(PATH, 'config/ca-vent'))
restart = RestartModel(config='ca-vent', population='dict')
model = restart.model

burn_df = model.demand.demand_per_unit_map_dn_um.df
burn_df.index.name = "Population"

burn_rates = qgrid.show_grid(burn_df)

# Editable Burn Rates
Click on any of the cells below to adjust burn rate assumptions. When you're done, click the `Run Model` button and the updated model will be generated.

In [334]:
display(create_run_button("Run Model"))
widgets.VBox([burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [335]:
import ipywidgets as widgets
from scipy.stats import norm
import math

# get updated burn rates
model.demand.adjust_burn(burn_rates.get_changed_df().to_numpy())

# charting 
burn_chart = generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)

def triangular(a,b,c):
    return math.sqrt( ((a*a + b*b + c*c) - a*b - a*c - b*c) / 18 )

ca_epi_df = data.epi_ranges()
ca_epi_df.set_index('Model', drop=True, inplace=True)

ca_epi_df = ca_epi_df * 4.12
ca_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
ca_stdev = ca_epi_df.apply(
    lambda row: triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

ca_epi_df['Population Mean'] = ca_epi_df.mean(axis=1)

ca_epi_df['Population SD'] = ca_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_ca_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(ca_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (model.demand.demand_by_pop_per_person_pn_uc.array.T * ca_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.drop(['Ventilators'], axis=1, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    chart = generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
ca_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))

ca_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
ca_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
ca_out = widgets.interactive_output(display_ca_eoq, {'cr': ca_cr_slider, 'days': ca_day_slider})
widgets.VBox([ca_cr_slider, ca_day_slider, ca_out, burn_chart])

## OES

In [348]:
oes_restart = RestartModel(
    population='oes',
    state='California',
    subpop='healthcare')
oes_model = oes_restart.model

oes_burn_df = oes_model.demand.demand_per_unit_map_dn_um.df
oes_burn_df.index.name = "Population"
oes_burn_chart = generate_group_bar_legend(oes_model.demand.demand_per_unit_map_dn_um.df)

oes_burn_rates = qgrid.show_grid(oes_burn_df)

In [349]:
display(create_run_button("Run Model"))
widgets.VBox([oes_burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [346]:
oes_model.demand.adjust_burn(oes_burn_rates.get_changed_df().to_numpy())
oes_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    oes_model.inventory.set_average_orders_per_period(oes_model.demand.demand_by_popsum1_total_rp1n_tc)
    backstop = [backstop]
    oes_model.inventory.order(oes_model.inventory.inv_by_popsum1_total_rp1n_tc)
    oes_model.inventory.set_min_in_periods(backstop)
    display_stock(oes_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
oes_pop = format_population(to_sheet(oes_model.population.population_pP_tr.df))
oes_out = widgets.interactive_output(dashboard, {'backstop': oes_slider})

widgets.VBox([oes_slider, oes_out, oes_burn_chart])

In [351]:
jhu_config = set_config(os.path.join(PATH, 'config/jhu'))
jhu_restart = RestartModel(config='jhu', population='dict')
jhu_model = jhu_restart.model

jhu_burn_df = jhu_model.demand.demand_per_unit_map_dn_um.df
jhu_burn_df.index.name = "Population"
jhu_burn_chart = generate_group_bar_legend(jhu_model.demand.demand_per_unit_map_dn_um.df)

jhu_burn_rates = qgrid.show_grid(jhu_burn_df)

In [353]:
display(create_run_button("Run Model"))
widgets.VBox([jhu_burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [356]:
jhu_model.demand.adjust_burn(jhu_burn_rates.get_changed_df().to_numpy())

jhu_epi_df = data.epi_ranges()
jhu_epi_df.set_index('Model', drop=True, inplace=True)
jhu_epi_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)

# adjusting so it's in terms of population, since we're not doing vents here
jhu_epi_df = jhu_epi_df * 4.12
jhu_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
jhu_stdev = jhu_epi_df.apply(
    lambda row: triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

jhu_epi_df['Population Mean'] = jhu_epi_df.mean(axis=1)

jhu_epi_df['Population SD'] = jhu_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_jhu_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(jhu_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", jhu_config)
    preds_df.array = (jhu_model.demand.demand_by_pop_per_person_pn_uc.array.T * jhu_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.rename({'Glove Pairs': 'Gloves'}, axis=1, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    chart = generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
jhu_burn_sheet = format_population(to_sheet(jhu_model.demand.demand_per_unit_map_dn_um.df))

jhu_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
jhu_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
jhu_out = widgets.interactive_output(display_jhu_eoq, {'cr': jhu_cr_slider, 'days': jhu_day_slider})
widgets.VBox([jhu_cr_slider, jhu_day_slider, jhu_out, jhu_burn_chart])